In [13]:
# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn
import pandas as pd
from keras.models import Sequential,Model
from keras.layers import LSTM, Dense,GRU,RNN,SimpleRNN,Dropout,TimeDistributed,RepeatVector,Bidirectional,concatenate
from keras.models import load_model,Input
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.utils.vis_utils import plot_model


In [2]:
def load_data(mode='train'):
    """
    Function to (download and) load the MNIST data
    :param mode: train or test
    :return: images and the corresponding labels
    """
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    if mode == 'train':
        x_train, y_train, x_valid, y_valid = mnist.train.images, mnist.train.labels, \
                                             mnist.validation.images, mnist.validation.labels
        return x_train, y_train, x_valid, y_valid
    elif mode == 'test':
        x_test, y_test = mnist.test.images, mnist.test.labels
    return x_test, y_test

def randomize(x, y):
    """ Randomizes the order of data samples and their corresponding labels"""
    permutation = np.random.permutation(y.shape[0])
    shuffled_x = x[permutation, :]
    shuffled_y = y[permutation]
    return shuffled_x, shuffled_y

def get_next_batch(x, y, start, end):
    x_batch = x[start:end]
    y_batch = y[start:end]
    return x_batch, y_batch

In [34]:
x_train, y_train, x_valid, y_valid = load_data(mode='train')
x_train.shape,y_train.shape
#print("Size of:")
#print("- Training-set:\t\t{}".format(len(y_train)))
#print("- Validation-set:\t{}".format(len(y_valid)))
 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


((55000, 784), (55000, 10))

Multi Output Model  With Shared Layers

In [36]:
# creating model
inputs = Input(shape = (784,))
dense1 = Dense(512, activation = 'relu',name='dense1')(inputs)
dense2 = Dense(128, activation = 'relu',name='dense2')(dense1)
dense3 = Dense(32, activation = 'relu',name='dense3')(dense2)
 
# create classification output
classification_output = Dense(10, activation = 'softmax',name='classification_output')(dense3)
 
    
# use output from dense layer 3 to create autoencder output
up_dense1 = Dense(128, activation = 'relu',name='up_dense1')(dense3)
up_dense2 = Dense(512, activation = 'relu',name='up_dense2')(up_dense1)
decoded_outputs = Dense(784,name='decoded_outputs')(up_dense2)

    
model = Model(inputs, [classification_output,decoded_outputs])
model.summary()    
    


m = 256
n_epoch = 10
model.compile(optimizer='adam', loss=['categorical_crossentropy', 'mse'], loss_weights = [1.0, 0.5], metrics = ['accuracy'])
model.fit(x_train,[y_train, x_train], epochs=n_epoch, batch_size=m, shuffle=True)



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 784)          0                                            
__________________________________________________________________________________________________
dense1 (Dense)                  (None, 512)          401920      input_15[0][0]                   
__________________________________________________________________________________________________
dense2 (Dense)                  (None, 128)          65664       dense1[0][0]                     
__________________________________________________________________________________________________
dense3 (Dense)                  (None, 32)           4128        dense2[0][0]                     
__________________________________________________________________________________________________
up_dense1 

Multi Input Model with one output

In [103]:
# create model with two inputs
input_1 = Input(shape = (784,))
input_2 = Input(shape = (784,))
# concatenate both feature layers and define output layer after some dense layers
concat = concatenate([input_1,input_2])
dense1 = Dense(512, activation = 'relu')(concat)
dense2 = Dense(128, activation = 'relu')(dense1)
dense3 = Dense(32, activation = 'relu')(dense2)
output = Dense(10, activation = 'softmax')(dense3)
 
Multi_Input_Model = Model(inputs=[input_1,input_2],outputs=[output])    
Multi_Input_Model.summary()   
Multi_Input_Model.compile(optimizer='adam', loss=['categorical_crossentropy'], loss_weights = [1.0], metrics = ['accuracy'])
Multi_Input_Model.fit([x_train,x_train],y_train, epochs=n_epoch, batch_size=m, shuffle=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41 (InputLayer)           (None, 784)          0                                            
__________________________________________________________________________________________________
input_42 (InputLayer)           (None, 784)          0                                            
__________________________________________________________________________________________________
concatenate_13 (Concatenate)    (None, 1568)         0           input_41[0][0]                   
                                                                 input_42[0][0]                   
__________________________________________________________________________________________________
dense_105 (Dense)               (None, 512)          803328      concatenate_13[0][0]             
__________

Multi-input and multi-output models

In [35]:
# creating LSTM model for the first input


# Data Dimension
num_input = 28          # MNIST data input (image shape: 28x28)
timesteps = 28          # Timesteps
n_classes = 10          # Number of classes, one class per digit
 
#Hyperparameters    
learning_rate = 0.001 # The optimization initial learning rate
epochs = 3          # Total number of training epochs
batch_size = 100      # Training batch size
display_freq = 100    # Frequency of displaying the training results
Dropout_rate=0.1
#Network configuration
num_hidden_units = 128  # Number of hidden units of the RNN   


input_LSTM = Input(shape = (timesteps,num_input))
LSTM_1=LSTM(num_hidden_units,return_sequences=True,dropout=0.1,name='LSTM_1')(input_LSTM)
LSTM_2=LSTM(num_hidden_units,return_sequences=False,name='LSTM_2')(LSTM_1)
LSTM_output = Dense(10, activation = 'softmax',name='LSTM_output')(LSTM_2)
LSTM_model=Model(input_LSTM,LSTM_output)

In [36]:
# creating feedforward model for the second  input

input_feedforward = Input(shape = (784,))
dense1 = Dense(512, activation = 'relu',name='dense1')(input_feedforward)
dense2 = Dense(128, activation = 'relu',name='dense2')(dense1)
dense3 = Dense(32, activation = 'relu',name='dense3')(dense2)
feedforward_output = Dense(10, activation = 'softmax',name='feedforward_output')(dense3)
feedforward_model=Model(input_feedforward,feedforward_output)

In [37]:
intermediate_from_LSTM = LSTM_model.get_layer("LSTM_output").output
intermediate_from_feedforward = feedforward_model.get_layer("feedforward_output").output

In [44]:
# combine output from two models into new input layer
concat = concatenate([intermediate_from_LSTM,intermediate_from_feedforward])
combinedense1 = Dense(512, activation = 'relu',name='combinedense1')(concat)
combinedense2 = Dense(128, activation = 'relu',name='combinedense2')(combinedense1)
combinedense3 = Dense(32, activation = 'relu',name='combinedense3')(combinedense2)
model_output = Dense(10, activation = 'softmax',name='model_output')(combinedense3)
concatenate_input_model=Model(intermediate_from_LSTM,intermediate_from_feedforward,model_output)

In [45]:
Multi_Input_Multi_Output_Model = Model(inputs=[input_LSTM,input_feedforward],outputs=[LSTM_output,feedforward_output,model_output])    
Multi_Input_Multi_Output_Model.summary()   


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 784)          0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
dense1 (Dense)                  (None, 512)          401920      input_10[0][0]                   
__________________________________________________________________________________________________
LSTM_1 (LSTM)                   (None, 28, 128)      80384       input_9[0][0]                    
__________________________________________________________________________________________________
dense2 (De

In [46]:
Multi_Input_Multi_Output_Model.compile(optimizer='adam', loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'], loss_weights = [1.0,1.0,1.0], metrics = ['accuracy'])
Multi_Input_Multi_Output_Model.fit([x_train.reshape(-1,28,28),x_train],[y_train,y_train,y_train], epochs=epochs, batch_size=batch_size, shuffle=True)

Epoch 1/3
55000/55000 [==============================] - 44s 803us/step - loss: 0.3011 - LSTM_output_loss: 0.0865 - feedforward_output_loss: 0.0663 - model_output_loss: 0.1483 - LSTM_output_acc: 0.9729 - feedforward_output_acc: 0.9800 - model_output_acc: 0.9776
Epoch 2/3
55000/55000 [==============================] - 45s 814us/step - loss: 0.1604 - LSTM_output_loss: 0.0655 - feedforward_output_loss: 0.0490 - model_output_loss: 0.0459 - LSTM_output_acc: 0.9799 - feedforward_output_acc: 0.9848 - model_output_acc: 0.9886
Epoch 3/3
55000/55000 [==============================] - 44s 807us/step - loss: 0.1229 - LSTM_output_loss: 0.0529 - feedforward_output_loss: 0.0367 - model_output_loss: 0.0333 - LSTM_output_acc: 0.9836 - feedforward_output_acc: 0.9890 - model_output_acc: 0.9916
